### Imports

In [2]:
from requests import get
from bs4 import BeautifulSoup as bs
from emosent import get_emoji_sentiment_rank
import json
import pandas as pd
import numpy as np
import time
import nltk
import emoji
import re

### Make a list of reliable and trust-worthy accounts (usernames)

In [9]:
accounts = ['ridouane_erramdani', 'bouchra_ddeau', 'mustapha_swinga',
            'raghib.official', 'cbvpost', 'ghomari.salaheddine',
            'chouftv_official', 'goud.ma'
           ]

### Parse HTML pages and find data Form our dataset

In [3]:

columns = ['profile_name', 'username', 'followers_count', 'total_posts', 'title', 'likes', 'number of comments', 'comments']
data = []

for account in accounts:
    print("\nScrapping " + account + "'s page..")
    response = get('https://www.instagram.com/' + account + '/')

    page_code = bs(response.text, 'html.parser')
    scripts = page_code.find_all('script')

    data_script = scripts[4]

    content = data_script.contents[0]
    data_object = content[content.find('{"config"') : -1]
    data_json = json.loads(data_object)
    data_json = data_json['entry_data']['ProfilePage'][0]['graphql']['user']
    posts_json = data_json['edge_owner_to_timeline_media']['edges']

    for postIndex in range(len(posts_json)):
        print("\tScrapping post number " + str(postIndex + 1)  + "..")
        shortcode = posts_json[postIndex]['node']['shortcode']
        url_comment='https://www.instagram.com/graphql/query/?query_hash=bc3296d1ce80a24b1b6e40b1e72903f5&variables={"shortcode":"'+str(shortcode)+'","first":40}'
        comments_list = json.loads(get(url_comment).text)
        comments_number = comments_list['data']['shortcode_media']['edge_media_to_parent_comment']['count']

        comments = ''

        for index in range(comments_number):
            try:
                comments += "|"+comments_list['data']['shortcode_media']['edge_media_to_parent_comment']['edges'][index]['node']['text']
                index += 1
            except:
                break

        try:
            title = posts_json[postIndex]['node']['edge_media_to_caption']['edges'][0]['node']['text']
        except:
            title = ''

        data.append([data_json['full_name'], data_json['username'], data_json['edge_followed_by']['count'], 
                 data_json['edge_owner_to_timeline_media']['count'], 
                 title,
                 posts_json[postIndex]['node']['edge_media_preview_like']['count'], 
                 comments_number,
                 comments
                ])

        time.sleep(8)
    time.sleep(8)

dataset = pd.DataFrame(data = data, columns = columns)
print('Finished !')


Scrapping ridouane_erramdani's page..
	Scrapping post number 1..
	Scrapping post number 2..
	Scrapping post number 3..
	Scrapping post number 4..
	Scrapping post number 5..
	Scrapping post number 6..
	Scrapping post number 7..
	Scrapping post number 8..
	Scrapping post number 9..
	Scrapping post number 10..
	Scrapping post number 11..
	Scrapping post number 12..

Scrapping bouchra_ddeau's page..
	Scrapping post number 1..
	Scrapping post number 2..
	Scrapping post number 3..
	Scrapping post number 4..
	Scrapping post number 5..
	Scrapping post number 6..
	Scrapping post number 7..
	Scrapping post number 8..
	Scrapping post number 9..
	Scrapping post number 10..
	Scrapping post number 11..
	Scrapping post number 12..

Scrapping mustapha_swinga's page..
	Scrapping post number 1..
	Scrapping post number 2..
	Scrapping post number 3..
	Scrapping post number 4..
	Scrapping post number 5..
	Scrapping post number 6..
	Scrapping post number 7..
	Scrapping post number 8..
	Scrapping post numbe

### Saving Data

In [3]:
#dataset
#dataset.to_csv('full_dataset.csv', index=None)
dataset= pd.read_csv('full_dataset.csv')

In [11]:

liste = []
for i in range(len(dataset)):
    liste.extend(str(dataset.loc[i]['comments']).split('|'))
    
comments = pd.DataFrame(data=liste, columns=["comment"]).dropna(axis=0,how='any')
comments.to_csv('comments.csv', index=None)


In [8]:
dataset

,profile_name,username,followers_count,total_posts,title,likes,number of comments,comments
0,Ridouane Erramdani,ridouane_erramdani,937018,2149,NaN,6064,150,|الدول العربية الي نهار ومطال وهما يسبو فيها ه...
1,Ridouane Erramdani,ridouane_erramdani,937018,2149,الذكرى...\nفين كنتو عام المسيرة؟؟,18366,736,|مكنتش 😁|Hhh mama kant ba9a sghiwra salwtha ma...
2,Ridouane Erramdani,ridouane_erramdani,937018,2149,أثارت تصريحات الوزير الأول الجزائري السابق عبد...,4799,344,|الحمد الله يا ربي الله يجعلنا نتقدمو وهما تاب...
3,Ridouane Erramdani,ridouane_erramdani,937018,2149,بعث الملك محمد السادس برقية إلى الرئيس الجزائر...,3557,124,|Top ولا Flop \n رأي و رايكم اقدر انفع غيري و ...
4,Ridouane Erramdani,ridouane_erramdani,937018,2149,NaN,9987,418,|الفزع من غلاء الاسعار والاثمنة دالخضر هو ما ي...
...,...,...,...,...,...,...,...,...
91,GOUD,goud.ma,39793,359,شي ملايرية دكازا ضد گاع اجراءات السلطات مع هاد...,196,7,|وحنا مالنا|ook he hoo|و حنا مالنا 2|وحنى مالن...
92,GOUD,goud.ma,39793,359,تسطية بعينينها. تفاصيل الخبر كامل في ستوري\n\n...,276,5,|😂😂chhal fikom ta3 lkdoub|nokta dial 2020 🤣|😂😂...
93,GOUD,goud.ma,39793,359,نكتة دارت هاد الحالة\n\nرابط المقال في الستوري...,215,5,|هاد المجتمع ديالنا عااار على عاار|تدخل النياب...
94,GOUD,goud.ma,39793,359,قال الفنان المغربي نعمان الحلو أنه امتنع على أ...,380,2,|بلاصت ميحولها لصندوق كورونا كون غي فرقها على ...


In [114]:
pd.DataFrame(dataset['title']).dropna(axis=0,how='any').to_csv('descriptions.csv', index=None, header=["title"], sep=',')

### Sentiment Analysis

Essentially, sentiment analysis or sentiment classification fall into the broad category of text classification tasks where you are supplied with a phrase, or a list of phrases and your classifier is supposed to tell if the sentiment behind that is positive, negative or neutral.

- **Polarity** is a float value within the range [-1.0 to 1.0] where 0 indicates neutral, +1 indicates a very positive sentiment and -1 represents a very negative sentiment.

- **Subjectivity** is a float value within the range [0.0 to 1.0] where 0.0 is very objective and 1.0 is very subjective. The subjective sentence expresses some personal feelings, views, beliefs, opinions, allegations, desires, beliefs, suspicions, and speculations whereas Objective sentences are factual.

#### **Sentiment analysis using Emojii**

For this section we will use **emosent-py**, it is a Python utility package to get Sentiment Rankings for Unicode Emojis.
The complete listed of Emojis to Sentiment Ranking supported by this project can be found at [Emoji Sentiment Ranking v1.0](http://kt.ijs.si/data/Emoji_sentiment_ranking/).

#### installation
You can install emosent-py from using pip:
**`pip install emosent-py`**

##### **Usage**

`from emosent import get_emoji_sentiment_rank
get_emoji_sentiment_rank('😂')`

##### **Output**

`{'unicode_codepoint': '0x1f602',
 'occurrences': 14622,
 'position': 0.805100583,
 'negative': 0.3614,
 'neutral': 0.4163,
 'positive': 0.6845,
 'unicode_name': 'FACE WITH TEARS OF JOY',
 'unicode_block': 'Emoticons',
 'sentiment_score': 0.221}`

So, in order to know if our emojii is positive or negative or neutral, we need to compare its negativity, neutral and positivity rank and see what rank is the maximum of them all.

We will use **emoji** library to extract emojis from text

**Let's get all emojis for comments first**

flags emojis will cause some errors for 

In [12]:
comments["list of emojis"] = np.nan
comments["list of emojis"] = comments["list of emojis"].astype('object')

def extract_emojis(text):
    txt = list(text)
    emoji_list = []
    for word in txt:
        if word in emoji.UNICODE_EMOJI:
            emoji_list.append(word)
    return emoji_list

i = 0
for comment in comments['comment']:
    if len(extract_emojis(str(comment))) > 0:
        comments['list of emojis'].iloc[i] = extract_emojis(str(comment))
    i+=1

comments = comments.dropna(axis=0,how='any')
comments.reset_index(drop=True, inplace=True)
comments.to_csv('emoji_sentiment_analysis.csv', index=None)

In [13]:
comments.head(10)

,comment,list of emojis
0,الدول العربية الي نهار ومطال وهما يسبو فيها هم...,"[😂, 😂, 😂, 😂]"
1,منافقين الفلسطنين فين كانت 😂😂😂😂,"[😂, 😂, 😂, 😂]"
2,بالدعاء 😂😂😂,"[😂, 😂, 😂]"
3,❤️❤️,"[❤, ❤]"
4,فلسطين : 🙈🙉🙊,"[🙈, 🙉, 🙊]"
5,المغرب أول بلد عتارف بالولايات 🇲🇦❤️المتحدة، تا...,"[🇲, 🇦, ❤]"
6,شاهد فلسطيني يرفض حرق علم المملكة المغربية 😂😂😂...,"[😂, 😂, 😂, 😂, 😂, 🇲, 🇦, 🇲, 🇦, 🇲, 🇦, 🇲, 🇦]"
7,فينكم ا جيران 😒 ههههه دبا يقول ليكم نسيتونا,[😒]
8,الخليج ديما مع المغرب الحب لا ينتهي ❤️❤️❤️❤️❤️...,"[❤, ❤, ❤, ❤, ❤, ❤, ❤, ❤, ❤, ❤]"
9,صحراءنا مغربية مأصلة بحكم حقيقة التاريخ و امتد...,"[👍, 👍, 👍]"


##### **Now let's determine Emoji's sentiment ranking**

In [14]:
comments["sentiment ranking"] = np.nan
comments["sentiment ranking"] = comments["sentiment ranking"].astype('float64')

i = 0
for emos in comments['list of emojis']:
    rate = 0
    valid_len = 0
    if (len(emos) > 0):
        for emo in emos:
            if emo in emoji.UNICODE_EMOJI:
                try:
                    score = get_emoji_sentiment_rank(emo)
                    rate += score['sentiment_score']
                    valid_len += 1
                except:
                    continue
        rate = float(rate / len(emos))
        comments["sentiment ranking"].iloc[i] = rate
    i+=1

C:\Users\Achraf\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [15]:
comments.to_csv('emoji_sentiment_analysis.csv', index=None)

##### **Now let's determine if it's positive or negative or objective (or non conclusive)**

In [16]:
comments['classification'] = np.nan
comments["classification"] = comments["classification"].astype('object')

i = 0
for rank in comments['sentiment ranking']:
    if rank == 0:
        comments['classification'].iloc[i] = 'Objective / Non-conclusive'
    elif rank > 0:
        comments['classification'].iloc[i] = 'Positive'
    elif rank < 0:
        comments['classification'].iloc[i] = 'Negative'
    i += 1


In [17]:
comments

,comment,list of emojis,sentiment ranking,classification
0,الدول العربية الي نهار ومطال وهما يسبو فيها هم...,"[😂, 😂, 😂, 😂]",0.221000,Positive
1,منافقين الفلسطنين فين كانت 😂😂😂😂,"[😂, 😂, 😂, 😂]",0.221000,Positive
2,بالدعاء 😂😂😂,"[😂, 😂, 😂]",0.221000,Positive
3,❤️❤️,"[❤, ❤]",0.746000,Positive
4,فلسطين : 🙈🙉🙊,"[🙈, 🙉, 🙊]",0.411333,Positive
...,...,...,...,...
1335,داك الفراش ديال الحزايقية ماشي الملايرية 😂,[😂],0.221000,Positive
1336,😂😂chhal fikom ta3 lkdoub,"[😂, 😂]",0.221000,Positive
1337,nokta dial 2020 🤣,[🤣],0.000000,Objective / Non-conclusive
1338,😂😂😂😂😂الله الله,"[😂, 😂, 😂, 😂, 😂]",0.221000,Positive


In [18]:
comments.to_csv('emoji_sentiment_analysis.csv', index=None)

In [9]:
dataset.to_excel('datasel.xlsx')

In [19]:
comments.to_excel('comments.xlsx')